# Knowledge-aware entities

To enable easier data integration, metadata fields are curated against standardized vocabularies. This process is often manual and quite time consuming, as it requires to look up the metadata terms and determine which one to use.

With {class}`lamindb.Knowledge` ([bionty](https://lamin.ai/docs/bionty)), we offer lookups via tab completion of names.

In [ ]:
import lamindb as ln
import lamindb.schema as lns
import lamindb.knowledge as lnk

ln.nb.header()

## Knowledge lookup

For instance, you can retrieve the Cell Type ontology id of "gamma delta T cell":

In [ ]:
ct_lookup = lnk.CellType().lookup

In [ ]:
ct_lookup.gamma_delta_T_cell

## Knowledge aware schema tables

You can also directly create a record for the CellType table in lamindb:

In [ ]:
lns.bionty.CellType(ontology_id=ct_lookup.gamma_delta_T_cell)

Similarly, assay readout can be looked up:

```{note}

lookup namespace can only starts with a letter, so we add `LOOKUP_` to the terms that aren't letters
```

In [ ]:
lnk.lookup.readout.LOOKUP_10x_3_v1

## Curate and add table records to knowledge ware schema tables

Now fast-forward, assuming we've done the curation (here we load a curated dataset with annotated ontology terms), let's link all the biosamples in a cross-tissue scRNA-seq dataset:

In [ ]:
adata = ln.dev.datasets.anndata_human_immune_cells()

meta = adata.obs.drop_duplicates(subset=adata.obs.columns)
meta.shape

In [ ]:
meta.head()

Let's first add all the cell types:

`.curate` allows you to check the passed ids are present in the knowledge table, it returns a new DataFramed indexed with the curated ids and a boolean `__curated__` columns.

Here we saw all terms can be linked. 🎉

In [ ]:
celltype_curate = lnk.CellType().curate(meta, column="cell_type_ontology_term_id")

We can go ahead and create records of the CellType table:

In [ ]:
celltype_records = [
    lns.bionty.CellType(ontology_id=i) for i in celltype_curate.index.unique()
]

In [ ]:
celltype_records[:3]

We can do the same for tissues:

In [ ]:
tissue_curate = lnk.Tissue().curate(meta, column="tissue_ontology_term_id")

In [ ]:
tissue_records = [
    lns.bionty.Tissue(ontology_id=i) for i in tissue_curate.index.unique()
]

In [ ]:
tissue_records[:3]

Finally, let's add them to the database:

In [ ]:
ln.add(celltype_records)
ln.add(tissue_records);

Check they are in the database:

In [ ]:
ln.select(lns.bionty.Tissue, name="blood").one()